In [1]:
import cv2
import numpy as np

In [12]:
win_name = 'scanning'
img = cv2.imread('./img/paper.jpg')
rows, cols = img.shape[:2]
draw = img.copy()
pts_cnt = 0
pts = np.zeros((4, 2), dtype=np.float32)

In [13]:
# 마우스 이벤트 콜백 함수 구현
def onMouse(event, x, y, flags, param):
    global pts_cnt                           # 마우스 좌표 저장
    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(draw, (x, y), 10, (0, 255, 0), -1)  # 죄표에 초록색 동그라미 표시
        cv2.imshow(win_name, draw)
        
        pts[pts_cnt] = [x, y]                 # 마우스 좌표 저장
        pts_cnt += 1
        
        # 좌표가 4개 수집됨
        if pts_cnt == 4:
            # 죄표 4개 중 상하좌우 찾기
            sm = pts.sum(axis=1)              # 4쌍의 좌표 각각 x+y 계산
            diff = np.diff(pts, axis = 1)     # 4쌍의 좌표 각각 x-y 계산
            
            topLeft = pts[np.argmin(sm)]      # x+y가 가장 작은 값이 좌상단 좌표
            bottomRight = pts[np.argmax(sm)]  # x+y가 가장 큰 값이 우하단 좌표
            topRight = pts[np.argmin(diff)]   # x-y가 가장 작은 값이 우상단 좌표
            bottomLeft = pts[np.argmax(diff)] # x-y가 가장 큰 값이 좌하단 좌표
            
            # 변환 전 4개의 좌표
            pts1 = np.float32([topLeft, topRight, bottomRight, bottomLeft])
            
            # 변환 후 영상에 사용할 서류의 폭과 높이 계산
            w1 = abs(bottomRight[0] - bottomLeft[0])    # 하단 좌우 좌표 거리
            w2 = abs(topRight[0] - topLeft[0])          # 상단 좌우 좌표 거리
            h1 = abs(topRight[1] - bottomRight[1])      # 우측 상하 좌표 거리
            h2 = abs(topLeft[1] - bottomLeft[1])        # 좌측 상하 좌표 거리
            width = max([w1, w2])                       # 두 좌우 거리 간의 최대 값이 서류의 폭
            height = max([h1, h2])                      # 두 상하 거리 간의 최대 값이 서류의 높이
            
            # 변환 후 4개의 좌표
            pts2 = np.float32([[0, 0], [width-1, 0], [width-1, height-1], [0, height-1]])
            
            # 변환행렬 계산
            mtrx = cv2.getPerspectiveTransform(pts1, pts2)
            
            # 원근 변환 적용
            result = cv2.warpPerspective(img, mtrx, (int(width), int(height)))
            
            cv2.imshow('scanned', result)

In [14]:
# 결과 출력
cv2.imshow(win_name, img)
cv2.setMouseCallback(win_name, onMouse)
cv2.waitKey(0)
cv2.destroyAllWindows()